# AwID: BFGS i Gradient Descent

In [128]:
using Zygote
using LinearAlgebra
using BenchmarkTools
@info "Done"

┌ Info: Done
└ @ Main In[128]:4


## BFGS

### Funkcje pomocnicze

In [129]:
function BacktrackingLineSearch(f, x, p; α=0.99, τ=0.95, c=0.3, maxIterations=1000)::Float64
    m = f'(x)'p
    t = -c * m
    i = 0
    while i < maxIterations
        if f(x) - f(x + α * p) >= α * t
            break
        end
        α *= τ
        i += 1
    end
    return α
end

BacktrackingLineSearch (generic function with 1 method)

In [130]:
function mulVectorByItsTranspose!(vector, preallocatedMatrix)
    preallocatedMatrix .*= 0
    for i in 1:length(vector)
        for j in 1:length(vector)
            preallocatedMatrix[i, j] = vector[i] * vector[j]
        end
    end
    return preallocatedMatrix
end 

mulVectorByItsTranspose! (generic function with 1 method)

In [131]:
function mulVectors!(v, y, preallocatedMatrix)
    n = length(v)
    for i = 1:n, j = 1:n
        preallocatedMatrix[i, j] = v[i] * y[j]
    end
    return preallocatedMatrix
end

mulVectors! (generic function with 1 method)

In [132]:
function sub!(x, y, preallocated)
    preallocated .= 0
    preallocated .+= x
    preallocated .-= y
    return preallocated
end

sub! (generic function with 1 method)

In [133]:
function add!(x, y, preallocated)
    preallocated .*= 0
    preallocated .+= x
    preallocated .+= y
    return preallocated
end

add! (generic function with 1 method)

### Implementacja

In [175]:
function Bfgs(f, x; maxIterations=10^5, ϵ::Float64=1e-6, maxLineSearchIterations::Int64=10000)
    ∇f = f'(x)
    B = zeros(length(x), length(x))
    for i=1:length(x)
        B[i, i] = 1
    end
    i = 0
    n = length(x)
    y = zeros(n)
    yyT = zeros(n, n)
    BS = zeros(n, 1)
    BSBST = zeros(n, n)
    p = zeros(n)
    s = zeros(n)
    reachedLimitOfIterations = true
    while i < maxIterations
        p = B \ -∇f # tu są 3 alokacje
        mul!(s, BacktrackingLineSearch(f, x, p; maxIterations=maxLineSearchIterations), p)
        if s's < ϵ
            reachedLimitOfIterations = false
            break
        end

        x .+= s
        ∇fn = f'(x)
        sub!(∇fn, ∇f, y) # y = ∇fn - ∇f
        ∇f = ∇fn
        mul!(BS, B, s)
        B .+= rdiv!(mulVectorByItsTranspose!(y, yyT), y's) 
        B .-=  mul!(BSBST, BS, BS') ./ s'BS
        i += 1
    end
    if(reachedLimitOfIterations) 
        @warn "Reached limit of iterations"
    end
    return x
end

Bfgs (generic function with 1 method)

## Gradient Descent

In [174]:
function GradientDescent(f, X; maxIterations = 1e5, α = 0.1, divideAlphaIfFIncreasesBy = 2.0, ϵ = 1e-6)
    Xₙ = X
    
    reachedLimitOfIterations = true    
    for _ in 1:maxIterations
        ∇f = f'(X)
        
        derivativesBellowTolerance = 0
        for derivative in ∇f    
            if abs(derivative) <= ϵ  * α
                derivativesBellowTolerance += 1
            end
        end
        if derivativesBellowTolerance == length(X)
                reachedLimitOfIterations = false
                break
        end
        fₙ = f(X)        
        X .-= α * ∇f
        fₙ₊₁ = f(X)

        if fₙ₊₁ >= fₙ && α > 1e-7
            α /= divideAlphaIfFIncreasesBy
            @debug "Decresing α, now: $(α)"
            continue
        end
        
    end
    
    if reachedLimitOfIterations
        @warn "GD: Reached limit of iterations"
    end
    
    return X;    
end

GradientDescent (generic function with 1 method)

### Gradient Descent z prealokacjami

In [173]:
function GradientDescentWithPrealocations(f, X; 
    maxIterations = 1e5, α = 0.1, divideAlphaIfFIncreasesBy = 2.0, ϵ = 1e-6
)
    reachedLimitOfIterations = true    
    preallocatedGradient = zero(X)
    
    iter = 0
    while iter <= maxIterations
        ∇f = f'(X)
        
        derivativesBellowTolerance = 0
        for derivative in ∇f    
            if abs(derivative) <= ϵ  * α
                derivativesBellowTolerance += 1
            end
        end
        if derivativesBellowTolerance == length(X)
            reachedLimitOfIterations = false
            break
        end
        
        fₙ = f(X)
        for i in 1:length(X)
            X[i] -= ∇f[i] * α
        end
        fₙ₊₁ = f(X)

        if fₙ₊₁ >= fₙ && α > 1e-7
            α /= divideAlphaIfFIncreasesBy
            @debug "Decresing α, now: $(α)"
            continue
        end
        iter +=1
        
    end
    
    if reachedLimitOfIterations
        @warn "GD: Reached limit of iterations"
    end
    
    return X;    
end

GradientDescentWithPrealocations (generic function with 1 method)

## Testy

### Funkcja z jednym argumentem

In [176]:
# ENV["JULIA_DEBUG"] = "warn"
f = (x) -> (x[1] + 15)^2
f(10)
println(@time GradientDescent(f , [4000.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400))
println(@time GradientDescentWithPrealocations(f, [40000.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400))
println(@time Bfgs(f, [4000.0]; maxIterations=10^3))

  0.083016 seconds (395.61 k allocations: 21.868 MiB, 99.94% compilation time)
[-14.999999779863478]
  0.074253 seconds (295.45 k allocations: 16.536 MiB, 9.62% gc time, 99.95% compilation time)
[-15.000000390478343]
  0.128585 seconds (228.01 k allocations: 11.616 MiB, 99.94% compilation time)
[-15.000015365720028]


In [181]:
println("-----------------BFGS------------------")
@benchmark Bfgs(f, [4000.0]; maxIterations=10^3)


-----------------BFGS------------------


BenchmarkTools.Trial: 
  memory estimate:  7.69 KiB
  allocs estimate:  117
  --------------
  minimum time:     6.552 μs (0.00% GC)
  median time:      6.856 μs (0.00% GC)
  mean time:        7.754 μs (9.03% GC)
  maximum time:     933.177 μs (98.35% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [182]:
println("------------------GD-------------------")
@benchmark GradientDescent(f , [4000.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400)

------------------GD-------------------


BenchmarkTools.Trial: 
  memory estimate:  24.58 KiB
  allocs estimate:  343
  --------------
  minimum time:     8.860 μs (0.00% GC)
  median time:      9.820 μs (0.00% GC)
  mean time:        12.731 μs (17.61% GC)
  maximum time:     4.582 ms (99.67% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [183]:
println("-----------------GDWP------------------")
@benchmark GradientDescentWithPrealocations(f, [40000.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400)


-----------------GDWP------------------


BenchmarkTools.Trial: 
  memory estimate:  14.73 KiB
  allocs estimate:  240
  --------------
  minimum time:     5.970 μs (0.00% GC)
  median time:      6.334 μs (0.00% GC)
  mean time:        8.038 μs (17.87% GC)
  maximum time:     994.806 μs (99.16% GC)
  --------------
  samples:          10000
  evals/sample:     5

### Rosenbrock


In [199]:
function rosenbrock(X)::Number
    result = 0
    for i in 1:length(X) - 1
        result += 100.0(X[i + 1] - X[i]^2) + (1 - X[i])
    end
    return result
end
rosenbrock([1, 1])

0.0

In [218]:
α=1.5e-6
ϵ=1e-3
maxIterations=10^4
X = rand(100)
X .*=400


println("-----------------------BFGS-----------------------")
copyOfX = copy(X)
resultBfgs = @time Bfgs(rosenbrock, copyOfX; 
    ϵ=ϵ, maxIterations=maxIterations
)
println("BFGS: ", rosenbrock(resultBfgs))

println("------------------------GD------------------------")
resultGradientDescent = @time GradientDescent(rosenbrock, copyOfX;
    α=α, ϵ=ϵ, maxIterations=maxIterations
)
println("GD  : ", rosenbrock(resultGradientDescent))
println("-----------------------GDWP-----------------------")
resultGradientDescent = @time GradientDescentWithPrealocations(rosenbrock, copyOfX;
    α=α, ϵ=ϵ, maxIterations=maxIterations
)
println("GDWP: ", rosenbrock(resultGradientDescent))


-----------------------BFGS-----------------------
  0.001842 seconds (28.07 k allocations: 4.249 MiB)
BFGS: -1.9467570982076227e13
------------------------GD------------------------
  3.244199 seconds (64.95 M allocations: 6.554 GiB, 12.58% gc time)
GD  : -7.846714865707695e15
-----------------------GDWP-----------------------
  3.296398 seconds (69.92 M allocations: 6.620 GiB, 12.66% gc time)
GDWP: -3.1646414611254943e18


In [219]:
# Wyłączym warningi bo będą przeszkadzać w benchmarkach
using Logging
Logging.disable_logging(Logging.Warn)

LogLevel(1001)

In [220]:
println("------------------------GD------------------------")
@benchmark GradientDescent(rosenbrock, XX; α=α, ϵ=ϵ, maxIterations=maxIterations) setup=(XX  = copy(X))

------------------------GD------------------------


BenchmarkTools.Trial: 
  memory estimate:  6.55 GiB
  allocs estimate:  64950005
  --------------
  minimum time:     3.183 s (12.75% GC)
  median time:      3.200 s (12.64% GC)
  mean time:        3.200 s (12.64% GC)
  maximum time:     3.218 s (12.54% GC)
  --------------
  samples:          2
  evals/sample:     1

In [226]:
println("-----------------------GDWP-----------------------")
@benchmark GradientDescentWithPrealocations(rosenbrock, XX; α=α, ϵ=ϵ, maxIterations=maxIterations) setup=(XX  = copy(X))

-----------------------GDWP-----------------------


BenchmarkTools.Trial: 
  memory estimate:  6.62 GiB
  allocs estimate:  69916997
  --------------
  minimum time:     3.261 s (12.68% GC)
  median time:      3.266 s (12.64% GC)
  mean time:        3.266 s (12.64% GC)
  maximum time:     3.271 s (12.60% GC)
  --------------
  samples:          2
  evals/sample:     1

In [222]:
println("-----------------------BFGS-----------------------")
@benchmark Bfgs(rosenbrock, XX; ϵ=ϵ, maxIterations=maxIterations) setup=(XX  = copy(X))

-----------------------BFGS-----------------------


BenchmarkTools.Trial: 
  memory estimate:  4.25 MiB
  allocs estimate:  28065
  --------------
  minimum time:     1.704 ms (0.00% GC)
  median time:      1.766 ms (0.00% GC)
  mean time:        2.003 ms (11.20% GC)
  maximum time:     4.510 ms (60.35% GC)
  --------------
  samples:          2489
  evals/sample:     1

### Suma kwadratów

In [189]:
function sumOfSquares(X)
    result = 0
    for x in X
        result += x^2
    end
    return result
end
sumOfSquares([1,2])

5

In [194]:
println(@time GradientDescent(sumOfSquares , [4000.0, -125.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400))
println(@time GradientDescentWithPrealocations(sumOfSquares, [4000.0, -125.0]; maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400))
println(@time Bfgs(sumOfSquares, [4000.0, -125.0]; maxIterations=10^3))

  0.002821 seconds (20.25 k allocations: 646.438 KiB)
[-3.8989172291113334e-7, 1.2184116340972917e-8]
  0.002803 seconds (20.90 k allocations: 644.812 KiB)
[-3.8989172291113334e-7, 1.2184116340972917e-8]
  0.000395 seconds (2.23 k allocations: 74.328 KiB)
[-1.530831385040737e-5, 4.78384814823037e-7]


In [195]:
@benchmark GradientDescent(sumOfSquares , [4000.0, -125.0]; 
    maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400)

BenchmarkTools.Trial: 
  memory estimate:  646.44 KiB
  allocs estimate:  20245
  --------------
  minimum time:     2.738 ms (0.00% GC)
  median time:      2.804 ms (0.00% GC)
  mean time:        2.887 ms (2.73% GC)
  maximum time:     8.550 ms (65.98% GC)
  --------------
  samples:          1732
  evals/sample:     1

In [196]:
@benchmark GradientDescentWithPrealocations(sumOfSquares, [4000.0, -125.0]; 
    maxIterations=10^3, divideAlphaIfFIncreasesBy=2, α=400)


BenchmarkTools.Trial: 
  memory estimate:  644.81 KiB
  allocs estimate:  20899
  --------------
  minimum time:     2.681 ms (0.00% GC)
  median time:      2.768 ms (0.00% GC)
  mean time:        2.850 ms (2.68% GC)
  maximum time:     8.362 ms (66.02% GC)
  --------------
  samples:          1753
  evals/sample:     1

In [197]:
@benchmark Bfgs(sumOfSquares, [4000.0, -125.0]; maxIterations=10^3)

BenchmarkTools.Trial: 
  memory estimate:  74.33 KiB
  allocs estimate:  2225
  --------------
  minimum time:     307.723 μs (0.00% GC)
  median time:      321.604 μs (0.00% GC)
  mean time:        331.760 μs (2.71% GC)
  maximum time:     6.130 ms (93.13% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Bibliografia

- https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm
- https://en.wikipedia.org/wiki/Backtracking_line_search
- https://en.wikipedia.org/wiki/Test_functions_for_optimization

